In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
from bs4 import BeautifulSoup
%matplotlib inline

In [2]:
# Get URL of the scraped page
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

In [3]:
# Generate Soup object
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
# Parse the webpage
results = soup.find(id='content').find(id = 'bodyContent').find(id = 'mw-content-text')
results = results.find('div', class_ = 'mw-parser-output')
results = results.find('table', class_ = 'wikitable sortable')

In [5]:
# Get the column headers
header = []
for item in results.find_all('th'):
    header.append(item.text.strip())
header[1] = 'Borough'
header

['Postal Code', 'Borough', 'Neighbourhood']

In [6]:
# Get the content of the table
content = []
for item in results.find_all('tr'):
    a = item.find_all('td')
    if not a:
        continue
    else:
        for x in a:
            b = [x.text.strip() for x in a]
            content.append(b)

In [7]:
# Create a new pandas dataframe for the table
tb = pd.DataFrame(content, columns = header)
tb.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1A,Not assigned,Not assigned
2,M1A,Not assigned,Not assigned
3,M2A,Not assigned,Not assigned
4,M2A,Not assigned,Not assigned


In [8]:
# Remove the "Not assigned" Borough
tb_ = tb[tb['Borough'] != "Not assigned"].drop_duplicates().reset_index(drop = True)
tb_

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
tb_.shape

(103, 3)

#### The package does not work for me
import geocoder

postal_code = tb_['Postal Code'].tolist()
lat = []
long = []
for item in postal_code:

    lat_lng_coords = None

    while(lat_lng_coords is None):
        g= geocoder.google('{}, Toronto, Ontario'.format(item))
        lat_lng_coords = g.latlng

    lat.append(lat_lng_coords[0])
    long.append(lat_lng_coords[1])
    
print(lat)

In [10]:
coord_data = pd.read_csv(r'/Users/chenli/Documents/Online Courses/IBM Data Science/Capstone Project/Geospatial_Coordinates.csv')
coord_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
main = tb_.merge(coord_data, on = 'Postal Code', how = 'left')
print(main.shape)
main.head()

(103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Explore and cluster the neighborhoods in Toronto

In [12]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [13]:
# Slice the dataset to create a new dataframe of Toronto data
toronto_data = main[main['Borough'].str.contains('Toronto', regex = False)].reset_index(drop=True)
toronto_data.shape

(39, 5)

In [14]:
# Create map of Toronto using Lat and Lon
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent = 'tor_explorer')
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude

map_toronto = folium.Map(location=[lat, lon], zoom_start = 11)

# Add markers to map
for lat, lon, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)

map_toronto


In [15]:
# Define Foursquare credentials and version
CLIENT_ID = '1OMUO421LFE4O0KAB24TS2HOMUR5GEKK05COF305DGOGZDOO'
CLIENT_SECRET = '0UKT4WPNSLDYGPYFI1M2DW11SYO4LUU3HJ021H4PMUT1BAEZ' # your Foursquare Secret
VERSION = '20200907' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1OMUO421LFE4O0KAB24TS2HOMUR5GEKK05COF305DGOGZDOO
CLIENT_SECRET:0UKT4WPNSLDYGPYFI1M2DW11SYO4LUU3HJ021H4PMUT1BAEZ


In [16]:
LIMIT = 100
radius = 500

In [17]:
def getNearbyVenues(names, latitudes, l n ongitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


#### Since I like to study at coffee shops, I want to find out which neighbourhood has the coffee shops

In [19]:
venues_grouped = toronto_venues.groupby(['Neighbourhood','Venue Category'])[['Venue']].count().reset_index()
venues_grouped[venues_grouped['Venue Category'] == 'Bakery'].reset_index(drop = \
                        True).sort_values(by = 'Venue', ascending = False).reset_index(drop =\
                        True).head()

,Neighbourhood,Venue Category,Venue
0,"Regent Park, Harbourfront",Bakery,3
1,Berczy Park,Bakery,2
2,"Kensington Market, Chinatown, Grange Park",Bakery,2
3,Studio District,Bakery,2
4,Stn A PO Boxes,Bakery,2


#### Therefore, Regent Park Harbourfront is my favorite neighbourhood based on my preference.

## Cluster similar neighbourhoods in Toronto

In [20]:
# One hot encoding
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep = "")
toronto_venues_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
col = [toronto_venues_onehot.columns[-1]] + list(toronto_venues_onehot.columns[:-1])
toronto_venues_onehot = toronto_venues_onehot[col]
toronto_venues_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_venues_onehot_clustering = toronto_venues_onehot.drop('Neighbourhood', 1)
toronto_venues_onehot_clustering.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Run k-means to cluster the neighborhood into 3 clusters.

In [22]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_venues_onehot_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [23]:
toronto_venues_onehot.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data.join(toronto_venues_onehot.set_index('Neighbourhood'), on = 'Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# create map
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters